### We are going to use a data set of a collection of randomised case-control studies of the effectiveness of descriptive social norms on hotel customers behavior to reuse their towels.

In [55]:
# libraries bambu and pms
import pandas as pd
import numpy as np
import bambi as bmb

In [56]:

data_file = "/Users/mexmex/Documents/3-Math_LU/HT2025/BERN02/Exercise 6/Data/towelData.csv"
data = pd.read_csv(data_file, sep=';', encoding='latin1')
count = data.iloc[:, -1] # get the last column with numbers or yes/no

# count has the number of yes and no for control and social norm groups
control_yes = count[::4].to_numpy() # every 4th starting from 0 - control group + yes
control_no = count[2::4].to_numpy() # every 4th starting from 2 - control group + no
control_total = np.array([y + n for y, n in zip(control_yes, control_no)])

social_yes = count[1::4].to_numpy() # every 4th starting from 1 - social norm group + yes
social_no = count[3::4].to_numpy() # every 4th starting from 3 - social norm group + no
social_total = np.array([y + n for y, n in zip(social_yes, social_no)])

study = np.arange(1,len(control_yes)+1) # 7 diferent studies

control_data = pd.DataFrame({"reuse": control_yes, "total": control_total, "group": "control", "study": study})
social_data = pd.DataFrame({ "reuse": social_yes, "total": social_total, "group": "social", "study": study})

combined_data = pd.concat([control_data, social_data], ignore_index=True)

# Convert data types - important for bambi that they are correctly set 
# can give errors otherwise
combined_data['reuse'] = combined_data['reuse'].astype(int)
combined_data['total'] = combined_data['total'].astype(int)
combined_data['group'] = combined_data['group'].astype('category')
combined_data['study'] = combined_data['study'].astype('category')

In [57]:
combined_data

,reuse,total,group,study
0,74,211,control,1
1,103,277,control,2
2,77,135,control,3
3,82,187,control,4
4,21,25,control,5
5,123,147,control,6
6,28,30,control,7
7,98,222,social,1
8,587,1318,social,2
9,406,655,social,3


In [58]:
# reuse is response Y_i. discrete binomial
# Model p_ij = intercept  + group_j + beta_1 * study
# x_ij is study
# Y_i - Bin(n_i, p_i)
# p_i Beta(a,b)
RANDOM_SEED = 48

### Estimate parameters using Bayesian inference

In [59]:
model0 = bmb.Model("p(reuse, total) ~ group + (1|study)", combined_data, family="binomial")
model0

       Formula: p(reuse, total) ~ group + (1|study)
        Family: binomial
          Link: p = logit
  Observations: 14
        Priors: 
    target = p
        Common-level effects
            Intercept ~ Normal(mu: 0.0, sigma: 1.5)
            group ~ Normal(mu: 0.0, sigma: 1.0)
        
        Group-level effects
            1|study ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 2.5495))

In [60]:
idata_model0 = model0.fit(tune=2000, draws=2000, target_accept=0.95,random_seed=RANDOM_SEED)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, group, 1|study_sigma, 1|study_offset]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/rich/live.py:256: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 10 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


### Make a summary of the parameter values by the posterior mean and 90% probability interval.

In [61]:
import arviz as az

az.summary(idata_model0, hdi_prob=0.90)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,0.427,0.425,-0.285,1.082,0.011,0.008,1599.0,2497.0,1.0
group[social],0.208,0.078,0.081,0.336,0.001,0.001,4551.0,4247.0,1.0
1|study_sigma,1.114,0.412,0.537,1.679,0.011,0.012,1492.0,2655.0,1.0
1|study[1],-0.943,0.429,-1.586,-0.202,0.011,0.008,1618.0,2699.0,1.0
1|study[2],-0.868,0.423,-1.546,-0.175,0.011,0.008,1599.0,2530.0,1.0
1|study[3],-0.143,0.426,-0.826,0.558,0.011,0.008,1610.0,2498.0,1.0
1|study[4],-0.638,0.425,-1.282,0.101,0.011,0.008,1629.0,2590.0,1.0
1|study[5],1.130,0.543,0.238,2.008,0.012,0.008,2173.0,2990.0,1.0
1|study[6],0.939,0.428,0.264,1.643,0.011,0.008,1625.0,2598.0,1.0
1|study[7],0.749,0.454,-0.001,1.468,0.011,0.007,1787.0,2840.0,1.0


### 5. Formulate hypotheses for the test of the effectiveness of the intervention referring to one or several parameters within the model.

$$
\begin{align}
H_0: & group_i <= 0 \\

H_1: & group_i > 0
\end{align}

$$

$$i = 1, 2 (controll, social)$$

### 6. Test the hypothesis using Bayesian inference.

In [62]:
posterior_group = idata_model0.posterior["group"].values.flatten()
print("Bayesian p-value: ",np.mean(posterior_group < 0))
# p-value very small, reject H0 meaning that group is statistically significant

Bayesian p-value:  0.002875
